In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

In [44]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [55]:
from langsmith import traceable
from langchain_core.prompts import ChatPromptTemplate

@traceable(run_type="retriever")
def fake_db_retrieval_step(question):
    with open('polly_facts.txt', 'r') as file:
        polly_facts = file.read()
    return {"question": question, "facts": polly_facts}

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a parrot named Polly! Here are some facts about yourself: {facts}\n Respond to questions about yourself based on those facts, and always repeat the user's question back before you respond."),
    ("user", "{question}")
])

chain = fake_db_retrieval_step | prompt | llm

In [ ]:
question = "Do you like animals?"
chain.invoke(question)

### Evaluation

In [48]:
from langsmith.schemas import Example, Run

def did_polly_respond(root_run: Run, example: Example) -> dict:
    print(root_run.outputs["output"])
    score = len(root_run.outputs["output"].pretty_repr()) > 0
    return {"key": "did_respond", "score": int(score)}

In [ ]:
from langsmith import evaluate

evaluate(
    chain.invoke,
    data="polly-ground-truth-qa",
    evaluators=[did_polly_respond],
    experiment_prefix="gpt-4o-mini"
)